In [ ]:
import tensorflow as tf
from zipfile import ZipFile
import glob
import cv2
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm_notebook as tqdm
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense,MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten

In [ ]:
import keras
from keras.preprocessing import image
from keras.preprocessing.image import load_img
import matplotlib.pyplot as plt

In [ ]:
 import os
 os.environ["KAGGLE_USERNAME"] = "write here"
 os.environ["KAGGLE_KEY"] = "write here"

 !kaggle datasets download sartajbhuvaji/brain-tumor-classification-mri

In [ ]:
!mkdir brain_tumor_detection

In [ ]:
from IPython.display import clear_output

!unzip brain-tumor-classification-mri.zip -d brain_tumor_detection

clear_output()

In [ ]:
def yes():
    for i in tqdm(os.listdir()):
      img = cv2.imread(i)
      img = cv2.resize(img,(224,224))
      X.append(img)
      y.append('0')
def no():
    for i in tqdm(os.listdir()):
      img = cv2.imread(i)
      img = cv2.resize(img,(224,224))
      X.append(img)
      y.append('1')

In [ ]:
X = []
y = []

os.chdir('/content/brain_tumor_detection/Training/glioma_tumor')
yes()
os.chdir('/content/brain_tumor_detection/Training/meningioma_tumor')
yes()
os.chdir('/content/brain_tumor_detection/Training/pituitary_tumor')
yes()
os.chdir('/content/brain_tumor_detection/Training/no_tumor')
no()

os.chdir('/content/brain_tumor_detection/Testing/glioma_tumor')
yes()
os.chdir('/content/brain_tumor_detection/Testing/meningioma_tumor')
yes()
os.chdir('/content/brain_tumor_detection/Testing/pituitary_tumor')
yes()
os.chdir('/content/brain_tumor_detection/Training/no_tumor')
no()


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
for i in range(4):
    plt.subplot(1, 4, i+1)
    plt.imshow(X[i], cmap="gray")
    plt.axis('off')
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=70)
print ("Shape of an image in X_train: ", X_train[0].shape)
print ("Shape of an image in X_test: ", X_test[0].shape)
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=2)
y_train = np.array(y_train)
X_train = np.array(X_train)
y_test = np.array(y_test)
X_test = np.array(X_test)
print("X_train Shape: ", X_train.shape)
print("X_test Shape: ", X_test.shape)
print("y_train Shape: ", y_train.shape)
print("y_test Shape: ", y_test.shape)

In [ ]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools
y = dict()
y[0] = []
y[1] = []
for set_name in (y_train, y_test):
    y[0].append(np.sum(set_name == 0))
    y[1].append(np.sum(set_name == 1))

trace0 = go.Bar(
    x=['Train Set', 'Test Set'],
    y=y[0],
    name='No',
    marker=dict(color='#33cc33'),
    opacity=0.7
)
trace1 = go.Bar(
    x=['Train Set', 'Test Set'],
    y=y[1],
    name='Yes',
    marker=dict(color='#ff3300'),
    opacity=0.7
)
data = [trace0, trace1]
layout = go.Layout(
    title='Count of classes in each set',
    xaxis={'title': 'Set'},
    yaxis={'title': 'Count'}
)
fig = go.Figure(data, layout)
iplot(fig)

In [ ]:
def plot_samples(X, y, labels_dict, n=50):
    """
    Creates a gridplot for desired number of images (n) from the specified set
    """
    for index in range(len(labels_dict)):
        imgs = X[np.argwhere(y == index)][:n]
        j = 10
        i = int(n/j)

        plt.figure(figsize=(15,6))
        c = 1
        for img in imgs:
            plt.subplot(i,j,c)
            plt.imshow(img[0])

            plt.xticks([])
            plt.yticks([])
            c += 1
        plt.suptitle('Tumor: {}'.format(labels_dict[index]))
        plt.show()

In [ ]:
from keras.applications import vgg16
from tensorflow import keras
img_rows, img_cols = 224, 224
vgg = vgg16.VGG16(weights = 'imagenet',include_top = False,input_shape = (img_rows, img_cols, 3) )

In [ ]:
for layer in vgg.layers:
    layer.trainable = False

for (i,layer) in enumerate(vgg.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

In [ ]:
def lw(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

from keras.models import Model

In [ ]:
num_classes = 2
FC_Head = lw(vgg, num_classes)
model = Model(inputs = vgg.input, outputs = FC_Head)
print(model.summary())

In [ ]:
from tensorflow.keras.models import Model
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])
history = model.fit(X_train,y_train,
                    epochs=10,
                    validation_data=(X_test,y_test))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from statistics import mean
acc = history.history['accuracy']
print("Training Accuracy : ",mean(acc))
val_acc = history.history['val_accuracy']
print("Validation Accuracy : ",mean(val_acc))
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
#Prediction for 1 image


img = image.load_img('/content/brain_tumor_detection/Testing/glioma_tumor/image(2).jpg', target_size=(224,224))
plt.figure(figsize=(10, 10))
plt.subplot(1,3,1)
plt.imshow(img, cmap="gray")
plt.axis('off')
plt.show
i = image.img_to_array(img)
input_arr = np.array([i])
predict_x=model.predict(input_arr)
class_x=np.argmax(predict_x)
print(predict_x)
print(class_x)
if(class_x == 0):
    print("The MRI Image is Of BRAIN TUMOR")
else:
    print("The MRI Image is Of HEALTHY BRAIN")